Bla bla
describe the pipeline for data prep

In [1]:
# Import all the necessary libraries
import xml.etree.ElementTree as ET
import numpy as np
# import csv - maybe not needed
import pandas as pd
import os
import csv

In [2]:
# Write a function for parsing frame timing from xml files
def caim_timing_ome_xml(ome_xml_file):
    # Parse the OME XML file with info about the timing
    # OME_XML_file_path = "D:/data/CaIm/" + exp_type + "/os" + str(mouse_id) + "/"
    # OME_XML_file_name = "os" + str(mouse_id) + "_" + exp_type + " " \

    tree = ET.parse(ome_xml_file)
    root = tree.getroot()

    # Namespace
    namespace = {"ome": "http://www.openmicroscopy.org/Schemas/OME/2011-06"}

    # Get all Plane elements
    planes = root.findall(".//ome:Plane", namespace)

    # Create empty lists to store the values of real times and frame numbers
    real_ts = []
    frame_ns = []

    # Extract real_ts and frame_ns values for each Plane
    for plane in planes:
        '''
        real_t = float(plane.attrib["DeltaT"])
        frame_n = int(plane.attrib["TheT"])
        real_ts.append(real_t)
        frame_ns.append(frame_n)
        '''
        real_ts.append(float(plane.attrib["DeltaT"]))
        frame_ns.append(int(plane.attrib["TheT"]))
        

    # Convert the lists to NumPy arrays
    # real_ts = int((real_ts - real_ts[0])*1000)
    real_ts_array = np.array(real_ts)*1000
  #  real_ts_array = (real_ts_array - real_ts[0])*1000
#real_ts_array = (real_ts_array - real_ts[0])*1000
    real_ts_array = real_ts_array.astype(int)
    frame_ns_array = np.array(frame_ns)

    # Combine DeltaT and TheT arrays as pairs
    frame_timing = list(zip(frame_ns_array, real_ts_array))

  
    # Return the NumPy arrays
    return frame_timing

In [24]:
# Mouse info
mouse_id = 652
exp_type = 'ambig'
# For ryzen, path:
path = 'd:/CaIm/' + exp_type + '/os' + str(mouse_id) + '/'
# For dell:
# path = 'c:/BG/data/os/CaIm/' + exp_type + '/os' + str(mouse_id) + '/'



In [25]:
# Read xml files for timing of the frames

filename_start = 'os' + str(mouse_id) + '_' + exp_type
filename_end = '.tif.xml'


#filename = path + 'os65_lawa - 1.tif.xml'
next_try = True; n_file = 1
frame_timing = []
if os.path.isfile(path + filename_start + filename_end):
    # proceed to parse it
    frame_timing = caim_timing_ome_xml(path + filename_start + filename_end)
elif os.path.isfile(path + filename_start + ' - ' + str(n_file) + filename_end):
    print('hop')
    while next_try:
        frame_timing_part = caim_timing_ome_xml(path + filename_start + ' - ' + str(n_file) + filename_end)
        frame_timing.extend(frame_timing_part)
       # function_to_parse(path + filename_start + ' - ' + str(n_file) + filename_end)
        n_file = n_file + 1
        next_try = os.path.isfile(path + filename_start + ' - ' + str(n_file) + filename_end)
        print('hop' + str(n_file))
        
else:
    print('no xml files found')
        
with open(path + 'os' + str(mouse_id) + '_' + exp_type + '_frame_timing.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['frame_ns', 'real_ts'])  # Write column headers
    writer.writerows(frame_timing)  # Write data rows

hop
hop2
hop3


In [2]:
# Data from video; frame rate, 1st blue light frame, 1st yellow light frame
video_timings = {'lawa':{}, 'discr':{}, 'ambig':{}}

video_timings['lawa'][65] = [30, 168, 500]
video_timings['lawa'][66] = [30, 484, 411]
video_timings['lawa'][67] = [30, 201, 586]
video_timings['lawa'][70] = [30, 248, 537]
video_timings['lawa'][71] = [30, 93, 443]
video_timings['lawa'][72] = [30, 171, 447]
video_timings['lawa'][73] = [30, 187, 462]

video_timings['discr'][65] = [30, 184, 275]
video_timings['discr'][66] = [30, 118, 212]
video_timings['discr'][70] = [30, 109, 563]
video_timings['discr'][73] = [30, 77, 407]


video_timings['ambig'][65] = [30, 334, 445]
video_timings['ambig'][652] = [30, 118, 212]

In [4]:
exp_type='discr'
mouse_id = 66
t_diff = round ((video_timings[exp_type][mouse_id][2] - video_timings[exp_type][mouse_id][1]) * \
(1000/video_timings[exp_type][mouse_id][0]))
t_diff

3133

# This one is for lawa, skip ahead for others

In [ ]:
behav = pd.read_csv(path + 'os' + str(mouse_id) + '_' + exp_type + '_behav.csv')
- behav['t_stim'][0] + t_diff

In [90]:
behav

,water_type,laser_type,t_port_move,t_stim
0,0,0,9885,4817
1,1,0,16406,11408
2,1,1,23168,18388
3,2,1,29737,25036
4,1,0,36589,31462
...,...,...,...,...
175,2,0,1206003,1201143
176,1,1,1212606,1207736
177,1,1,1219654,1214519
178,2,0,1226295,1221549


In [ ]:
# Change the timings in all the behavioral files
t_diff = round ((video_timings[exp_type][mouse_id][2] - video_timings[exp_type][mouse_id][1]) * \
(1000/video_timings[exp_type][mouse_id][0]))

adj_time =  - behav['t_stim'][0] + t_diff;
t_licks_L = pd.read_csv(path + 'os' + str(mouse_id) + '_' + exp_type + '_licks_L.csv', header=None)
t_licks_R = pd.read_csv(path + 'os' + str(mouse_id) + '_' + exp_type + '_licks_R.csv', header=None)

t_licks_L = t_licks_L  + adj_time
t_licks_R = t_licks_R  + adj_time
behav['t_port_move'] = behav['t_port_move'] + adj_time
behav['t_stim'] = behav['t_stim'] + adj_time


behav.to_csv(path + 'os' + str(mouse_id) + '_' + exp_type + '_behav.csv', index=False)
t_licks_L.to_csv(path + 'os' + str(mouse_id) + '_' + exp_type + '_t_licks_L.csv', index=False, header=None)
t_licks_R.to_csv(path + 'os' + str(mouse_id) + '_' + exp_type + '_t_licks_R.csv', index=False, header=None)

In [92]:
behav

,water_type,laser_type,t_port_move,t_stim
0,0,0,14701,9633
1,1,0,21222,16224
2,1,1,27984,23204
3,2,1,34553,29852
4,1,0,41405,36278
...,...,...,...,...
175,2,0,1210819,1205959
176,1,1,1217422,1212552
177,1,1,1224470,1219335
178,2,0,1231111,1226365


# This one is for discr and mb for ambig

In [9]:
path = 'D:\\OneDrive - Rutgers University\\CaIm\\discr\\'
path

'D:\\OneDrive - Rutgers University\\CaIm\\discr\\'

In [37]:
mouse_id = 73

In [5]:
dates = {66: '2023-06-23', 70: '2023-06-28', 73: '2023-06-29'}

In [38]:
path_exp = path + 'os' + str(mouse_id) + '\\' + dates[mouse_id] + '\\' + 'os' + str(mouse_id) + '_' + exp_type + '_' + dates[mouse_id] 
path_behav = path_exp  + '_behav.csv'
path_licks_L = path_exp  + '_licks_L.csv'
path_licks_R = path_exp  + '_licks_R.csv'

In [39]:
behav = pd.read_csv(path_behav)
licks_L = pd.read_csv(path_licks_L, header=None)
licks_R = pd.read_csv(path_licks_R, header=None)

In [40]:
licks_L

,0
0,3206
1,3557
2,4511
3,5064
4,10225
...,...
329,1066795
330,1068069
331,1070894
332,1121642


In [41]:
# in case the values for timing are negative, correct that
if behav['tone_ts'][0] < 0:
    behav['tone_ts'] = - behav['tone_ts']
  #  t_licks_L = - t_licks_L
  #  t_licks_R = - t_licks_R
behav

,missed,laser,side,tone_ts,choice,free
0,0,0,1,2139,1,0
1,0,1,-1,5814,-1,0
2,0,0,-1,9856,1,0
3,0,0,-1,14493,-1,0
4,0,0,1,18215,1,0
...,...,...,...,...,...,...
295,0,0,1,1217713,-1,0
296,1,2,1,1222759,0,0
297,1,2,1,1227325,0,0
298,1,0,-1,1231857,0,0


In [42]:
# Change the timings in all the behavioral files

t_diff = round ((video_timings[exp_type][mouse_id][2] - video_timings[exp_type][mouse_id][1]) * \
(1000/video_timings[exp_type][mouse_id][0]))

adj_time =  - behav['tone_ts'][0] + t_diff;

licks_L = licks_L  + adj_time
licks_R = licks_R  + adj_time

behav['tone_ts'] = behav['tone_ts'] + adj_time

behav.to_csv(path_behav, index=False)
licks_L.to_csv(path_licks_L, index=False, header=None)
licks_R.to_csv(path_licks_R, index=False, header=None)

In [43]:
licks_L

,0
0,12067
1,12418
2,13372
3,13925
4,19086
...,...
329,1075656
330,1076930
331,1079755
332,1130503


In [44]:
t_diff

11000

Correct timing based on video

In [24]:
# Data from video; frame rate, 1st blue light frame, 1st yellow light frame
video_timings = {'lawa':{}, 'discr':{}, 'ambig':{}}

video_timings['lawa'][65] = [30, 168, 500]
video_timings['lawa'][66] = [30, 484, 411]
video_timings['lawa'][67] = [30, 201, 586]
video_timings['lawa'][70] = [30, 248, 537]
video_timings['lawa'][72] = [30, 171, 447]
video_timings['lawa'][73] = [30, 187, 462]

video_timings['discr'][65] = [30, 184, 275]
video_timings['ambig'][65] = [30, 334, 445]

                            
# os65
# os65 lawa 05/05/2023
frame_rate = 30; frame_blue = 168; frame_yellow =  500; 
# os65 discr
frame_rate = 30; frame_blue = 184; frame_yellow =  275; 
# os65 ambig  V_20230609_175028_vHDR_Auto 
frame_rate = 30; frame_blue = 334; frame_yellow = 445

# os 66 lawa
frame_rate = 30; frame_blue = 484; frame_yellow = 411

# os67 lawa
frame_rate = 30; frame_blue = 201; frame_yellow = 586

# os70 lawa
frame_rate = 30; frame_blue = 248; frame_yellow = 537

# os72 lawa
frame_rate = 30; frame_blue = 171; frame_yellow = 447

# os73 lawa
frame_rate = 30; frame_blue = 187; frame_yellow = 462

SyntaxError: invalid syntax (4008084054.py, line 5)

In [ ]:
# Correct the timings for a given 